<a href="https://colab.research.google.com/github/Likhithluck/Capstone-Project/blob/tarun/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install pdf2image
# !pip install datasets
# !pip install pytesseract
# !pip install pdfplumber
# !apt-get install tesseract-ocr
# !apt-get install libtesseract-dev
# !apt-get install poppler-utils # Install poppler-utils which includes pdfinfo
# # pip install camelot-py[cv]

In [ ]:
import pandas as panda
from huggingface_hub import login
from datasets import load_dataset
from PIL import Image as img
import matplotlib.pyplot as plt
from google.colab import files
import cv2
import numpy as np
import pytesseract
from pdf2image import convert_from_path
import pdfplumber
from google.colab import files
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

In [ ]:
from google.colab import userdata
key=userdata.get('hugging_face_key')

In [ ]:
login(token=key)

In [ ]:
# dataset1 = load_dataset('mychen76/invoices-and-receipts_ocr_v1')
# dataset2 = load_dataset('mychen76/invoices-and-receipts_ocr_v2')

In [ ]:
# #Basic OCR
# # Preprocessing Function
# def preprocess_image(image):
#     gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)
#     denoised = cv2.fastNlMeansDenoising(gray, None, 30, 7, 21)
#     clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8, 8))
#     contrast_enhanced = clahe.apply(denoised)
#     kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
#     cleaned = cv2.morphologyEx(contrast_enhanced, cv2.MORPH_CLOSE, kernel)
#     thresh = cv2.adaptiveThreshold(cleaned, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
#                                    cv2.THRESH_BINARY, 11, 2)
#     return thresh


# def extract_text_from_pdf(pdf_path):
#     images = convert_from_path(pdf_path, dpi=300 ,poppler_path="/usr/bin" )

#     extracted_text = ""

#     for i, image in enumerate(images):
#         processed_image = preprocess_image(image)
#         custom_config = '--psm 6 --oem 3'
#         page_text = pytesseract.image_to_string(processed_image, config=custom_config)
#         extracted_text += f"\n--- Page {i + 1} ---\n{page_text}"

#     print("Extracted Text:\n", extracted_text)


# #Dataset pdf
# pdf_path = "/content/invoice_Yana Sorensen_5434.pdf"
# #OCR extraction
# text=extract_text_from_pdf(pdf_path)
# print(text)

In [ ]:

# def extract_clean_text(pdf_path):
#     clean_text = ""
#     with pdfplumber.open(pdf_path) as pdf:
#         for page_num, page in enumerate(pdf.pages):
#             text = page.extract_text()
#             if text:
#                 clean_text += text.strip() + "\n\n"
#     print(clean_text)
#     with open("clean_text_output.txt", "w", encoding="utf-8") as file:
#         file.write(clean_text)

# # Path to PDF
# pdf_path = "/content/invoice_Yana Sorensen_5434.pdf"

# # Run Extraction
# ocr_pdf_plum=extract_clean_text(pdf_path)


In [ ]:
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
import cv2
import numpy as np

#upload file
def upload_pdf():
    uploaded = files.upload()
    pdf_path = next(iter(uploaded))
    return pdf_path

#preprocess the image
def preprocess_image(image):
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)
    sharpened = cv2.filter2D(gray, -1, np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]))
    denoised = cv2.fastNlMeansDenoising(sharpened, None, 30, 7, 21)
    clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8, 8))
    contrast_enhanced = clahe.apply(denoised)
    return contrast_enhanced


#OCR Extraction
def extract_text_with_ocr(pdf_path):
    extracted_text = ""
    images = convert_from_path(pdf_path, dpi=300, poppler_path="/usr/bin")

    for i, image in enumerate(images):
        processed_image = preprocess_image(image)
        custom_config = '--psm 3 --oem 3'
        page_text = pytesseract.image_to_string(processed_image, config=custom_config)
        extracted_text += f"--- Page {i + 1} ---\n{page_text.strip()}\n\n"

    return extracted_text

# Table Extraction Using pdfplumber
def extract_tables_from_pdf(pdf_path):
    table_data = ""

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):

            # Adjust table settings
            tables = page.extract_tables(
                table_settings={
                    "snap_tolerance": 10,
                    "join_tolerance": 10,  # Merges fragmented rows
                    "edge_min_length": 50, # Filters out small text artifacts
                    "horizontal_strategy": "lines",  # Try extracting text as lines
                    "vertical_strategy": "lines",  # Strategy to align vertically
                }
            )


            if tables:
                table_data += f"\n--- Tables from Page {page_num + 1} ---\n"
                for table in tables:
                    for row in table:

                        cleaned_row = [str(cell).strip() if cell else '' for cell in row]
                        table_data += " | ".join(cleaned_row) + "\n"
            else:
                print("No table found on this page.")

    return table_data


def extract_data(pdf_path):
    print("(OCR)")
    ocr_text = extract_text_with_ocr(pdf_path)

    print("\nTables")
    table_data = extract_tables_from_pdf(pdf_path)

    return ocr_text, table_data


#Testing
# pdf_path = "/content/invoice_Yana Sorensen_5434.pdf"
pdf_path=upload_pdf()
ocr_text, table_data = extract_data(pdf_path)

print("\nOCR Text\n")
print(ocr_text)
print(table_data)


Saving invoice_Sung Shariari_26597.pdf to invoice_Sung Shariari_26597.pdf
(OCR)

Tables
No table found on this page.

OCR Text

--- Page 1 ---
Superstore INVOICE

# 26597
Date: Apr 10 2012
Bill To: Ship To:
. . ; Ship Mode: Standard Class
Sung Shariari Kendari, Sulawesi
Tenggara, Indonesia Balance Due: $4,984.80
Item | Quantity Bcc) Amount
Smead Shelving, Industrial 12 $492.12 $5,905.48
Storage, Office Supplies, OFF-ST-6058
Subtotal: $5,905.48
Discount (17%): $1,003.93
Shipping: $83.25
Total: $4,984.80

Notes:
Thanks for your business!

Terms:
Order ID : ID-2012-SS2087559-41009





In [ ]:
# Check the types of ocr_text and table_data
ocr_text_type = type(ocr_text)
table_data_type = type(table_data)

extracted_data = {
    'ocr_text': ocr_text,
    'table_data': table_data
}


In [ ]:
extracted_data

{'ocr_text': '--- Page 1 ---\nSuperstore INVOICE\n\n# 26597\nDate: Apr 10 2012\nBill To: Ship To:\n. . ; Ship Mode: Standard Class\nSung Shariari Kendari, Sulawesi\nTenggara, Indonesia Balance Due: $4,984.80\nItem | Quantity Bcc) Amount\nSmead Shelving, Industrial 12 $492.12 $5,905.48\nStorage, Office Supplies, OFF-ST-6058\nSubtotal: $5,905.48\nDiscount (17%): $1,003.93\nShipping: $83.25\nTotal: $4,984.80\n\nNotes:\nThanks for your business!\n\nTerms:\nOrder ID : ID-2012-SS2087559-41009\n\n',
 'table_data': ''}

In [ ]:
# # Example OCR text
# ocr_text = """
# Superstore INVOICE

# # 5434
# Date: May 31 2012
# Bill To: Ship To:
# Ship Mode: Standard Class
# Yana Sorensen Matagalpa,
# Matagalpa, Nicaragua Balance Due: $1,031.66
# Item | Quantity Bcc) Amount
# Dania Library with Doors, Metal 2 $482.48 $964.96
# Bookcases, Furniture, FUR-BO-3901
# Subtotal: $964.96
# Shipping: $66.70
# Total: $1,031.66

# Notes:
# Thanks for your business!

# Terms:
# Order ID : MX-2012-YS21 88093-41060
# """


In [ ]:
import spacy

# Load spaCy's pre-trained NER model
nlp = spacy.load("en_core_web_sm")

def perform_ner(text):
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        entities.append({
            "text": ent.text,
            "label": ent.label_,
            "start_char": ent.start_char,
            "end_char": ent.end_char
        })
    return entities


ner_results = perform_ner(ocr_text)


In [ ]:
import json
import re

def ocr_to_json_with_ner(ocr_text, ner_results):
    # Initialize variables
    invoice_data = {
        "invoice_number": None,
        "date": None,
        "bill_to": None,
        "ship_to": None,
        "ship_mode": None,
        "balance_due": None,
        "items": [],
        "subtotal": None,
        "discount": None,
        "shipping": None,
        "total": None,
        "notes": "",
        "terms": {"order_id": None}
    }

    # Extract entities and populate the JSON structure
    person_entities = [entity for entity in ner_results if entity["label"] == "PERSON"]
    # gpe_entities = [entity for entity in ner_results if entity["label"] == "GPE"]
    date_entities = [entity for entity in ner_results if entity["label"] == "DATE"]
    invoice_entities = [entity for entity in ner_results if entity["label"] == "MONEY"]

    # Assign the first PERSON entity to bill_to
    if len(person_entities) >= 1:
        invoice_data["bill_to"] = person_entities[2]["text"]
    # DATE
    if len(date_entities) >= 1:
        invoice_data["date"] = date_entities[0]["text"]
    # INVOICE NUM
    if len(invoice_entities) >= 1:
        invoice_data["invoice_number"] = invoice_entities[0]["text"]

    # Assign ship_to based on bill_to value
    if invoice_data["bill_to"]:
        # Search for the bill_to value in the OCR text
        bill_to_index = ocr_text.find(invoice_data["bill_to"])
        if bill_to_index != -1:
            # Extract the text after the bill_to value
            text_after_bill_to = ocr_text[bill_to_index + len(invoice_data["bill_to"]):]
            # Find the text before "Balance Due"
            balance_due_index = text_after_bill_to.find("Balance Due")
            if balance_due_index != -1:
                # Extract the ship_to value (text between the comma after bill_to and "Balance Due")
                ship_to_value = text_after_bill_to[:balance_due_index].strip()
                # Remove any leading/trailing punctuation
                ship_to_value = ship_to_value.strip(", ")
                # Clean the ship_to value by removing ": " and "\n"
                ship_to_value = ship_to_value.replace(": ", "").replace("\n", " ")
                invoice_data["ship_to"] = ship_to_value

    # Extract Ship Mode using regex
    ship_mode_match = re.search(r"Ship Mode: ([\w\s]+)\n", ocr_text)
    if ship_mode_match:
        ship_mode = ship_mode_match.group(1).strip()
        invoice_data["ship_mode"] = ship_mode

    # Extract Balance Due using regex
    balance_due_match = re.search(r"Balance Due: \$([\d,]+\.\d{2})", ocr_text)
    if balance_due_match:
        balance_due = balance_due_match.group(1).strip()
        invoice_data["balance_due"] = float(balance_due.replace(",", ""))

    # Extract Subtotal using regex
    subtotal_match = re.search(r"Subtotal: \$([\d,]+\.\d{2})", ocr_text)
    if subtotal_match:
      subtotal = subtotal_match.group(1).strip()
      invoice_data["subtotal"] = float(subtotal.replace(",", ""))

    # Extract Discount using regex (assuming discount is labeled as "Discount" or "Discount (20%)")
    discount_match = re.search(r"Discount \(?\d*%?\)?: \$([\d,]+\.\d{2})", ocr_text)
    if discount_match:
        discount = discount_match.group(1).strip()
        invoice_data["discount"] = float(discount.replace(",", ""))

    # Extract Shipping using regex
    shipping_match = re.search(r"Shipping: \$([\d,]+\.\d{2})", ocr_text)
    if shipping_match:
        shipping = shipping_match.group(1).strip()
        invoice_data["shipping"] = float(shipping.replace(",", ""))

    # Extract Total using regex
    total_match = re.search(r"Total: \$([\d,]+\.\d{2})", ocr_text)
    if total_match:
        total = total_match.group(1).strip()
        invoice_data["total"] = float(total.replace(",", ""))


    # Extract Notes (text after "Notes:" until the next "Terms:" or end of text)
    notes_match = re.search(r"Notes:\s*(.*?)\s*(?=Terms:|$)", ocr_text, re.DOTALL)
    if notes_match:
        notes = notes_match.group(1).strip()
        invoice_data["notes"] = notes

    # Extract Order ID (text after "Order ID :")
    order_id_match = re.search(r"Order ID : (\S+)", ocr_text)
    if order_id_match:
        order_id = order_id_match.group(1).strip()
        invoice_data["terms"]["order_id"] = order_id+date_entities[-1]["text"]



    return json.dumps({"invoice": invoice_data}, indent=4)



json_output = ocr_to_json_with_ner(ocr_text, ner_results)
print(json_output)

{
    "invoice": {
        "invoice_number": "4,984.80",
        "date": "10 2012",
        "bill_to": "Sung Shariari Kendari",
        "ship_to": "Sulawesi Tenggara, Indonesia",
        "ship_mode": "Standard Class",
        "balance_due": 4984.8,
        "items": [],
        "subtotal": 5905.48,
        "discount": 1003.93,
        "shipping": 83.25,
        "total": 4984.8,
        "notes": "Thanks for your business!",
        "terms": {
            "order_id": "ID-2012-SS2087559-4100941009"
        }
    }
}


In [ ]:
import re

def extract_item_details(ocr_text):
    # Initialize variables to hold item details
    items = []

    # Regex pattern to capture the item name, quantity, unit price, and total amount
    item_pattern = re.compile(r"([A-Za-z0-9\s,]+(?:,?\s?[\w\-]+)*)\s+(\d+)\s+\$([\d,]+\.\d{2})\s+\$([\d,]+\.\d{2})")

    # Find all matches of the item pattern
    matches = item_pattern.findall(ocr_text)

    # Process each match into a dictionary and append to the items list
    for match in matches:
        item_name = match[0].strip()
        quantity = match[1]
        unit_price = match[2]
        total_amount = match[3]
        items.append({
            "Item Name": item_name,
            "Quantity": quantity,
            "Unit Price": unit_price,
            "Total Amount": total_amount
        })

    return items


# Example OCR text
ocr_text = """Bill To: Ship To:
. . ; Ship Mode: Standard Class
Sung Shariari Kendari, Sulawesi
Tenggara, Indonesia Balance Due: $4,984.80
Item | Quantity Bcc) Amount
Smead Shelving, Industrial 12 $492.12 $5,905.48
Storage, Office Supplies, OFF-ST-6058
Subtotal: $5,905.48
Discount (17%): $1,003.93
Shipping: $83.25"""

# Call the function to extract item details
items = extract_item_details(ocr_text)

# Print the extracted item details
for item in items:
    print(f"Item Name: {item['Item Name']}")
    print(f"Quantity: {item['Quantity']}")
    print(f"Unit Price: {item['Unit Price']}")
    print(f"Total Amount: {item['Total Amount']}")
